In [1]:
import numpy as np
import timeit


M = np.array([[1, 3, 3, 6], [4, 2, 8, 2], [3, 3, 4, 5], [2, 6, 3, 1]])

# Compute the inverse
N = np.linalg.inv(M)

# Matrix multiplication
result1 = np.matmul(M, N)
result2 = np.matmul(N, M)

# Check if the results are close to the identity matrix
identity_matrix = np.eye(4)
print("M * N:")
print(result1)
print("N * M:")
print(result2)



# Divide M and N into submatrices
n = M.shape[0] // 2
A = M[:n, :n]
B = M[:n, n:]
C = M[n:, :n]
D = M[n:, n:]

E = N[:n, :n]
F = N[:n, n:]
G = N[n:, :n]
H = N[n:, n:]

# Strassen's algorithm for matrix multiplication
def strassen_multiply(A, B):
    if A.shape[0] <= 128:  # Use traditional matrix multiplication for small matrices
        return np.matmul(A, B)
    
    n = A.shape[0] // 2
    
    A11, A12 = A[:n, :n], A[:n, n:]
    A21, A22 = A[n:, :n], A[n:, n:]
    B11, B12 = B[:n, :n], B[:n, n:]
    B21, B22 = B[n:, :n], B[n:, n:]
    
    P1 = strassen_multiply(A11 + A22, B11 + B22)
    P2 = strassen_multiply(A21 + A22, B11)
    P3 = strassen_multiply(A11, B12 - B22)
    P4 = strassen_multiply(A22, B21 - B11)
    P5 = strassen_multiply(A11 + A12, B22)
    P6 = strassen_multiply(A21 - A11, B11 + B12)
    P7 = strassen_multiply(A12 - A22, B21 + B22)
    
    C11 = P1 + P4 - P5 + P7
    C12 = P3 + P5
    C21 = P2 + P4
    C22 = P1 - P2 + P3 + P6
    
    return np.vstack((np.hstack((C11, C12)), np.hstack((C21, C22))))

# Perform matrix multiplication using Strassen's algorithm
result3 = strassen_multiply(M, N)
result4 = strassen_multiply(N, M)

print("M * N using Strassen's algorithm:")
print(result3)
print("N * M using Strassen's algorithm:")
print(result4)




# Time for traditional matrix multiplication
time_basic = timeit.timeit("np.matmul(M, N)", globals=globals(), number=1000)

# Time for Strassen's algorithm
time_strassen = timeit.timeit("strassen_multiply(M, N)", globals=globals(), number=1000)

print("Execution time for basic matrix multiplication:", time_basic)
print("Execution time for Strassen's algorithm:", time_strassen)




M * N:
[[ 1.00000000e+00  1.38777878e-17 -5.55111512e-17 -2.77555756e-17]
 [-2.77555756e-17  1.00000000e+00  5.55111512e-17  0.00000000e+00]
 [-6.93889390e-17 -6.93889390e-18  1.00000000e+00 -4.16333634e-17]
 [ 4.16333634e-17  4.85722573e-17  8.32667268e-17  1.00000000e+00]]
N * M:
[[ 1.00000000e+00  2.91433544e-16  4.78783679e-16  3.81639165e-16]
 [ 0.00000000e+00  1.00000000e+00  2.77555756e-17  5.55111512e-17]
 [-9.71445147e-17  4.16333634e-17  1.00000000e+00 -2.08166817e-17]
 [-5.55111512e-17 -2.77555756e-17 -6.93889390e-17  1.00000000e+00]]
M * N using Strassen's algorithm:
[[ 1.00000000e+00  1.38777878e-17 -5.55111512e-17 -2.77555756e-17]
 [-2.77555756e-17  1.00000000e+00  5.55111512e-17  0.00000000e+00]
 [-6.93889390e-17 -6.93889390e-18  1.00000000e+00 -4.16333634e-17]
 [ 4.16333634e-17  4.85722573e-17  8.32667268e-17  1.00000000e+00]]
N * M using Strassen's algorithm:
[[ 1.00000000e+00  2.91433544e-16  4.78783679e-16  3.81639165e-16]
 [ 0.00000000e+00  1.00000000e+00  2.7755575

In [5]:
def strassen_multiply(matrix_A, matrix_B):
    # Base case: if the matrices are 1x1
    if len(matrix_A) == 1:
        return [[matrix_A[0][0] * matrix_B[0][0]]]

    # Calculate the split index
    split_index = len(matrix_A) // 2

    # Splitting the matrices into smaller submatrices
    a00 = [matrix_A[i][:split_index] for i in range(split_index)]
    a01 = [matrix_A[i][split_index:] for i in range(split_index)]
    a10 = [matrix_A[i][:split_index] for i in range(split_index, len(matrix_A))]
    a11 = [matrix_A[i][split_index:] for i in range(split_index, len(matrix_A))]

    b00 = [matrix_B[i][:split_index] for i in range(split_index)]
    b01 = [matrix_B[i][split_index:] for i in range(split_index)]
    b10 = [matrix_B[i][:split_index] for i in range(split_index, len(matrix_B))]
    b11 = [matrix_B[i][split_index:] for i in range(split_index, len(matrix_B))]

    # Recursive calculations of the seven products (P1 to P7)
    P1 = strassen_multiply(a00, subtract_matrices(b01, b11))
    P2 = strassen_multiply(add_matrices(a00, a01), b11)
    P3 = strassen_multiply(add_matrices(a10, a11), b00)
    P4 = strassen_multiply(a11, subtract_matrices(b10, b00))
    P5 = strassen_multiply(add_matrices(a00, a11), add_matrices(b00, b11))
    P6 = strassen_multiply(subtract_matrices(a01, a11), add_matrices(b10, b11))
    P7 = strassen_multiply(subtract_matrices(a00, a10), add_matrices(b00, b01))

    # Calculate the resulting submatrices (C00, C01, C10, C11)
    C00 = add_matrices(subtract_matrices(add_matrices(P5, P4), P2), P6)
    C01 = add_matrices(P1, P2)
    C10 = add_matrices(P3, P4)
    C11 = subtract_matrices(subtract_matrices(add_matrices(P5, P1), P3), P7)

    # Combine the resulting submatrices to form the result matrix C
    result_matrix = [C00[i] + C01[i] for i in range(len(C00))]
    result_matrix += [C10[i] + C11[i] for i in range(len(C10))]

    return result_matrix

def add_matrices(matrix_A, matrix_B):
    return [[matrix_A[i][j] + matrix_B[i][j] for j in range(len(matrix_A[0]))] for i in range(len(matrix_A))]

def subtract_matrices(matrix_A, matrix_B):
    return [[matrix_A[i][j] - matrix_B[i][j] for j in range(len(matrix_A[0]))] for i in range(len(matrix_A))]

# Test matrices
matrix_X = [[3, 2],
            [4, 8]]

matrix_Y = [[1, 5],
            [9, 6]]

# Perform matrix multiplication using Strassen's algorithm
result_matrix = strassen_multiply(matrix_X, matrix_Y)

# Print the result
for row in result_matrix:
    print(row)


[21, 27]
[76, 68]
